# BNP Paribas Securities Services - Category Analysis

**Problématique** : *How can we improve our institutional clients' day-to-day experience?*

**Période d'analyse** : Janvier à Septembre 2024-2025

**Approche** :
1. Analyse de la répartition par catégorie de demandes
2. Temps de traitement par catégorie (impact sur l'expérience client)
3. Tendances temporelles et patterns
4. Identification des pain points clients
5. KPIs

---
## 1. Configuration & Connexion

In [25]:
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration
BASE_DIR = Path('/Users/jo/Desktop/BNP Paribas')
DB_PATH = BASE_DIR / 'Data/Processed/hobart_database.db'
EXPORT_DIR = BASE_DIR / 'Data/Exports'
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# Connexion
conn = sqlite3.connect(str(DB_PATH))

print("✅ Connexion établie à la base de données Hobart")
print(f"📂 Base: {DB_PATH}")
print(f"📊 Période: Janvier - Septembre 2024-2025")

# Configuration Plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

✅ Connexion établie à la base de données Hobart
📂 Base: /Users/jo/Desktop/BNP Paribas/Data/Processed/hobart_database.db
📊 Période: Janvier - Septembre 2024-2025


---
## 2. Vue d'Ensemble : Statistiques Clés

Comprendre le volume et la nature des demandes clients.

In [26]:
# Statistiques globales (Jan-Sep uniquement, exclure Octobre incomplet)
stats_global = pd.read_sql_query("""
SELECT 
    COUNT(*) as total_srs,
    COUNT(CASE WHEN CLOSINGDATE IS NOT NULL THEN 1 END) as srs_closed,
    ROUND(COUNT(CASE WHEN CLOSINGDATE IS NOT NULL THEN 1 END) * 100.0 / COUNT(*), 2) as closure_rate,
    COUNT(DISTINCT CATEGORY_ID) as categories_count,
    COUNT(DISTINCT JUR_DESK_ID) as desks_count,
    ROUND(AVG(CAST((julianday(CLOSINGDATE) - julianday(CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close
FROM sr
WHERE strftime('%Y-%m', CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
""", conn)

print("="*70)
print("📊 VUE D'ENSEMBLE (Janvier - Septembre)")
print("="*70)
print(f"\n📋 Service Requests totaux : {stats_global['total_srs'][0]:,}")
print(f"✅ SRs fermés : {stats_global['srs_closed'][0]:,} ({stats_global['closure_rate'][0]}%)")
print(f"📂 Catégories de demandes : {stats_global['categories_count'][0]}")
print(f"👥 Desks actifs : {stats_global['desks_count'][0]}")
print(f"⏱️  Temps moyen de résolution : {stats_global['avg_hours_to_close'][0]:.1f} heures ({stats_global['avg_hours_to_close'][0]/24:.1f} jours)")

display(stats_global)

📊 VUE D'ENSEMBLE (Janvier - Septembre)

📋 Service Requests totaux : 4,145,682
✅ SRs fermés : 4,116,982 (99.31%)
📂 Catégories de demandes : 1547
👥 Desks actifs : 759
⏱️  Temps moyen de résolution : 143.4 heures (6.0 jours)


,total_srs,srs_closed,closure_rate,categories_count,desks_count,avg_hours_to_close
0,4145682,4116982,99.31,1547,759,143.45


---
## 3. Analyse par Catégorie : Répartition des Demandes Clients

**Question Clé** : Quelles sont les catégories de demandes les plus fréquentes ?

In [27]:
# Répartition par catégorie (Top 20)
category_distribution = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    c.ID as category_id,
    COUNT(sr.ID) as total_requests,
    COUNT(CASE WHEN sr.CLOSINGDATE IS NOT NULL THEN 1 END) as closed_requests,
    ROUND(COUNT(CASE WHEN sr.CLOSINGDATE IS NOT NULL THEN 1 END) * 100.0 / COUNT(sr.ID), 2) as closure_rate,
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close,
    ROUND(COUNT(sr.ID) * 100.0 / (SELECT COUNT(*) FROM sr WHERE strftime('%Y-%m', CREATIONDATE) BETWEEN '2024-01' AND '2025-09'), 2) as pct_of_total
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY c.ID, c.NAME
ORDER BY total_requests DESC
LIMIT 20
""", conn)

print("="*70)
print("📂 TOP 20 CATÉGORIES DE DEMANDES CLIENTS")
print("="*70)
print(f"\nTop 5 catégories représentent {category_distribution.head(5)['pct_of_total'].sum():.1f}% du volume total\n")

display(category_distribution)

📂 TOP 20 CATÉGORIES DE DEMANDES CLIENTS

Top 5 catégories représentent 10.4% du volume total



,category_name,category_id,total_requests,closed_requests,closure_rate,avg_hours_to_close,pct_of_total
0,Tax,172255,103527,100680,97.25,457.93,2.50
1,Cash instruction,353585,91925,91822,99.89,53.72,2.22
2,BAU Asset Creation,218638,82196,82171,99.97,21.78,1.98
3,CREST,371413,76884,76847,99.95,28.73,1.85
4,Others,154192,75916,75748,99.78,72.00,1.83
5,Investigation Level 1,219273,71533,71499,99.95,65.01,1.73
6,Settlement,173197,65129,64965,99.75,116.30,1.57
7,FRPP,327439,63285,63229,99.91,65.41,1.53
8,ESMX,327466,60882,60837,99.93,26.19,1.47
9,DEFF,327494,58473,58446,99.95,38.88,1.41


In [28]:
# Visualisation : Répartition par catégorie
fig = px.bar(
    category_distribution.head(15),
    x='category_name',
    y='total_requests',
    title='Top 15 Catégories de Demandes Clients (Jan-Sep)',
    labels={'category_name': 'Catégorie', 'total_requests': 'Nombre de Demandes'},
    text='total_requests',
    color='closure_rate',
    color_continuous_scale='RdYlGn',
    hover_data=['avg_hours_to_close', 'pct_of_total']
)

fig.update_traces(textposition='outside')
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

---
## 4. Temps de Traitement par Catégorie : Impact sur l'Expérience Client

**Question Clé** : Quelles catégories ont les temps de traitement les plus longs (pain points clients) ?

In [29]:
# Temps de traitement par catégorie (catégories avec > 100 SRs)
treatment_time_by_category = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    COUNT(sr.ID) as total_requests,
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours,
    ROUND(MIN(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as min_hours,
    ROUND(MAX(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as max_hours,
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)) / 24, 1) as avg_days
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE sr.CLOSINGDATE IS NOT NULL 
  AND sr.CREATIONDATE IS NOT NULL
  AND strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY c.NAME
HAVING COUNT(sr.ID) > 100
ORDER BY avg_hours DESC
LIMIT 20
""", conn)

print("="*70)
print("⏱️  TEMPS DE TRAITEMENT PAR CATÉGORIE (Top 20 plus longs)")
print("="*70)
print("\n⚠️  Catégories avec les temps les plus longs = Pain points clients !\n")

display(treatment_time_by_category)

⏱️  TEMPS DE TRAITEMENT PAR CATÉGORIE (Top 20 plus longs)

⚠️  Catégories avec les temps les plus longs = Pain points clients !



,category_name,total_requests,avg_hours,min_hours,max_hours,avg_days
0,Change And Transitions,108,2933.13,0.42,13034.54,122.2
1,EXTERNAL QUERIES,7664,2487.25,0.00,13965.28,103.6
2,Post-Reporting 1042-S,106,2088.03,0.01,12568.41,87.0
3,CLIENT PROJECT,282,1905.83,0.01,10219.10,79.4
4,FATCA & CRS,131,1836.68,0.00,14184.59,76.5
5,Matrix tool,176,1775.24,2.03,12402.02,74.0
6,Tax Voucher,172,1668.21,0.01,11419.93,69.5
7,INTERNAL QUERIES,13680,1572.94,0.00,15426.08,65.5
8,Tax Authorities,134,1532.72,0.01,10462.41,63.9
9,General client queries,197,1497.96,0.01,12984.61,62.4


In [30]:
# Visualisation : Temps moyen par catégorie
fig = go.Figure()

# Trier par temps de traitement
df_sorted = treatment_time_by_category.head(15).sort_values('avg_days')

fig.add_trace(go.Bar(
    y=df_sorted['category_name'],
    x=df_sorted['avg_days'],
    orientation='h',
    text=df_sorted['avg_days'].apply(lambda x: f"{x:.1f}j"),
    textposition='outside',
    marker_color=df_sorted['avg_days'],
    marker_colorscale='Reds',
    hovertemplate='<b>%{y}</b><br>Temps moyen: %{x:.1f} jours<extra></extra>'
))

fig.update_layout(
    title='Temps Moyen de Traitement par Catégorie (en jours)',
    xaxis_title='Jours',
    yaxis_title='Catégorie',
    height=600,
    showlegend=False
)

fig.show()

---
## 5. Tendances Temporelles : Évolution des Demandes par Catégorie

**Question Clé** : Comment évoluent les demandes clients dans le temps ?

In [31]:
# Évolution mensuelle des top 10 catégories
top_10_categories = category_distribution.head(10)['category_id'].tolist()

monthly_category_trends = pd.read_sql_query(f"""
SELECT 
    strftime('%Y-%m', sr.CREATIONDATE) as month,
    c.NAME as category_name,
    COUNT(sr.ID) as total_requests,
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE sr.CATEGORY_ID IN ({','.join(map(str, top_10_categories))})
  AND strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY month, c.NAME
ORDER BY month, total_requests DESC
""", conn)

print("="*70)
print("📈 ÉVOLUTION MENSUELLE DES TOP 10 CATÉGORIES")
print("="*70)
print(f"\nPériode : Janvier 2024 - Septembre 2025")
print(f"Nombre de mois analysés : {monthly_category_trends['month'].nunique()}\n")

display(monthly_category_trends.head(30))

📈 ÉVOLUTION MENSUELLE DES TOP 10 CATÉGORIES

Période : Janvier 2024 - Septembre 2025
Nombre de mois analysés : 21



,month,category_name,total_requests,avg_hours_to_close
0,2024-01,Tax,5096,754.05
1,2024-01,BAU Asset Creation,4230,32.57
2,2024-01,ESMX,3894,27.04
3,2024-01,Investigation Level 1,3872,60.46
4,2024-01,CREST,3828,28.16
5,2024-01,Others,3695,85.66
6,2024-01,FRPP,3314,89.90
7,2024-01,DEFF,2873,42.81
8,2024-01,Settlement,2531,121.20
9,2024-01,Cash instruction,202,104.30


In [32]:
# Visualisation : Évolution des top 5 catégories
top_5_categories_names = category_distribution.head(5)['category_name'].tolist()
df_top5 = monthly_category_trends[monthly_category_trends['category_name'].isin(top_5_categories_names)]

fig = px.line(
    df_top5,
    x='month',
    y='total_requests',
    color='category_name',
    title='Évolution Mensuelle des Top 5 Catégories de Demandes',
    labels={'month': 'Mois', 'total_requests': 'Nombre de Demandes', 'category_name': 'Catégorie'},
    markers=True
)

fig.update_layout(height=500, hovermode='x unified')
fig.show()

---
## 6. KPIs Orientés Expérience Client

**Focus** : SLA, First Response Time, Taux de Résolution

In [33]:
# KPIs expérience client par catégorie
client_experience_kpis = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    COUNT(sr.ID) as total_requests,
    
    -- Taux de résolution
    ROUND(COUNT(CASE WHEN sr.CLOSINGDATE IS NOT NULL THEN 1 END) * 100.0 / COUNT(sr.ID), 2) as resolution_rate,
    
    -- Temps de première réponse (Acknowledge)
    ROUND(AVG(CAST((julianday(sr.ACKNOWLEDGE_DATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_first_response_hours,
    
    -- Temps de traitement total
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_total_hours,
    
    -- Respect SLA (expiration_date)
    ROUND(COUNT(CASE WHEN sr.CLOSINGDATE <= sr.EXPIRATION_DATE THEN 1 END) * 100.0 / 
          COUNT(CASE WHEN sr.CLOSINGDATE IS NOT NULL AND sr.EXPIRATION_DATE IS NOT NULL THEN 1 END), 2) as sla_compliance,
    
    -- Nombre de contacts moyen
    ROUND(AVG((SELECT COUNT(*) FROM srcontact WHERE srcontact.SR_ID = sr.ID)), 2) as avg_contacts
    
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY c.NAME
HAVING COUNT(sr.ID) > 100
ORDER BY total_requests DESC
LIMIT 20
""", conn)

print("="*70)
print("🎯 KPIs EXPÉRIENCE CLIENT PAR CATÉGORIE")
print("="*70)
print("\nIndicateurs clés :")
print("  • Taux de résolution : % de SRs fermés")
print("  • Premier temps de réponse : Temps avant acknowledge")
print("  • SLA Compliance : % de SRs traités avant expiration")
print("  • Contacts moyens : Nombre d'interactions par SR\n")

display(client_experience_kpis)

🎯 KPIs EXPÉRIENCE CLIENT PAR CATÉGORIE

Indicateurs clés :
  • Taux de résolution : % de SRs fermés
  • Premier temps de réponse : Temps avant acknowledge
  • SLA Compliance : % de SRs traités avant expiration
  • Contacts moyens : Nombre d'interactions par SR



,category_name,total_requests,resolution_rate,avg_first_response_hours,avg_total_hours,sla_compliance,avg_contacts
0,Tax,141559,97.63,8.81,464.84,64.03,4.03
1,Cash instruction,91925,99.89,0.52,53.72,81.17,3.11
2,BAU Asset Creation,82196,99.97,0.03,21.78,34.13,1.54
3,CREST,76884,99.95,0.46,28.73,85.30,1.63
4,Others,75916,99.78,3.67,72.00,73.40,1.90
5,Investigation Level 1,71533,99.95,1.48,65.01,72.87,2.70
6,Settlement,70002,99.74,0.83,118.89,70.52,3.95
7,FRPP,63285,99.91,0.45,65.41,72.13,3.11
8,ESMX,60882,99.93,0.38,26.19,77.92,1.60
9,DEFF,58473,99.95,0.37,38.88,76.64,2.42


In [34]:
# Visualisation : SLA Compliance vs Volume (Top 15)
df_viz = client_experience_kpis.head(15).copy()

fig = px.scatter(
    df_viz,
    x='total_requests',
    y='sla_compliance',
    size='avg_total_hours',
    color='resolution_rate',
    hover_name='category_name',
    title='SLA Compliance vs Volume (Top 15 Catégories)',
    labels={
        'total_requests': 'Volume de Demandes',
        'sla_compliance': 'SLA Compliance (%)',
        'avg_total_hours': 'Temps Moyen (h)',
        'resolution_rate': 'Taux Résolution (%)'
    },
    color_continuous_scale='RdYlGn',
    size_max=60
)

fig.add_hline(y=95, line_dash="dash", line_color="red", 
              annotation_text="Target SLA 95%", annotation_position="right")

fig.update_layout(height=600)
fig.show()

---
## 7. Analyse des Pain Points Clients

**Identification** : Catégories avec faible performance sur l'expérience client

In [35]:
# Identifier les pain points (volume élevé + SLA faible OU temps long)
pain_points = client_experience_kpis[
    ((client_experience_kpis['total_requests'] > 1000) & 
     ((client_experience_kpis['sla_compliance'] < 90) | 
      (client_experience_kpis['avg_total_hours'] > 100)))
].sort_values('total_requests', ascending=False)

print("="*70)
print("⚠️  PAIN POINTS CLIENTS IDENTIFIÉS")
print("="*70)
print("\nCritères : Volume > 1000 ET (SLA < 90% OU Temps > 100h)")
print(f"\nNombre de catégories problématiques : {len(pain_points)}\n")

if len(pain_points) > 0:
    print("🔴 Catégories nécessitant une attention immédiate :\n")
    display(pain_points)
    
    # Calculer l'impact total
    total_impact = pain_points['total_requests'].sum()
    total_srs = category_distribution['total_requests'].sum()
    impact_pct = (total_impact / total_srs) * 100
    
    print(f"\n💥 Impact : {total_impact:,} SRs concernés ({impact_pct:.1f}% du volume total)")
else:
    print("✅ Aucun pain point majeur identifié avec ces critères")

⚠️  PAIN POINTS CLIENTS IDENTIFIÉS

Critères : Volume > 1000 ET (SLA < 90% OU Temps > 100h)

Nombre de catégories problématiques : 18

🔴 Catégories nécessitant une attention immédiate :



,category_name,total_requests,resolution_rate,avg_first_response_hours,avg_total_hours,sla_compliance,avg_contacts
0,Tax,141559,97.63,8.81,464.84,64.03,4.03
1,Cash instruction,91925,99.89,0.52,53.72,81.17,3.11
2,BAU Asset Creation,82196,99.97,0.03,21.78,34.13,1.54
3,CREST,76884,99.95,0.46,28.73,85.30,1.63
4,Others,75916,99.78,3.67,72.00,73.40,1.90
5,Investigation Level 1,71533,99.95,1.48,65.01,72.87,2.70
6,Settlement,70002,99.74,0.83,118.89,70.52,3.95
7,FRPP,63285,99.91,0.45,65.41,72.13,3.11
8,ESMX,60882,99.93,0.38,26.19,77.92,1.60
9,DEFF,58473,99.95,0.37,38.88,76.64,2.42



💥 Impact : 1,189,066 SRs concernés (96.2% du volume total)


---
## 8. Analyse des Réouvertures (Indicateur de Satisfaction)

**Question** : Quelles catégories ont le plus de SRs réouverts (insatisfaction client) ?

In [36]:
# Analyse des réouvertures par catégorie
reopened_analysis = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    COUNT(sr.ID) as total_requests,
    COUNT(CASE WHEN sr.REOPEN_DATE IS NOT NULL THEN 1 END) as reopened_count,
    ROUND(COUNT(CASE WHEN sr.REOPEN_DATE IS NOT NULL THEN 1 END) * 100.0 / COUNT(sr.ID), 2) as reopen_rate
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY c.NAME
HAVING COUNT(sr.ID) > 100 AND reopened_count > 0
ORDER BY reopen_rate DESC
LIMIT 20
""", conn)

print("="*70)
print("🔄 ANALYSE DES RÉOUVERTURES (Indicateur de Satisfaction)")
print("="*70)
print("\n⚠️  Taux de réouverture élevé = Résolution incomplète ou insatisfaction\n")

if len(reopened_analysis) > 0:
    display(reopened_analysis)
    
    avg_reopen_rate = reopened_analysis['reopen_rate'].mean()
    print(f"\n📊 Taux moyen de réouverture : {avg_reopen_rate:.2f}%")
else:
    print("✅ Très peu de réouvertures dans les données")

🔄 ANALYSE DES RÉOUVERTURES (Indicateur de Satisfaction)

⚠️  Taux de réouverture élevé = Résolution incomplète ou insatisfaction



,category_name,total_requests,reopened_count,reopen_rate
0,Breakdown RAS,785,782,99.62
1,Cannis,457,442,96.72
2,Processing,538,473,87.92
3,Trade Placement,3097,2532,81.76
4,Physicals,391,315,80.56
5,Requests to HUB,794,632,79.60
6,Support to Production,859,680,79.16
7,Aberdeen Case,192,146,76.04
8,OST,1829,1361,74.41
9,Settlement Trade Format,429,311,72.49



📊 Taux moyen de réouverture : 74.52%


---
## 9. nombre de SR par catégorie et combien d’entre elles ont eu besoin de faire une activité

In [37]:
# Analyse des SRs avec activités par catégorie
activity_analysis = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    COUNT(DISTINCT sr.ID) as total_srs,
    COUNT(DISTINCT CASE WHEN activity.ID IS NOT NULL THEN sr.ID END) as srs_with_activity,
    ROUND(COUNT(DISTINCT CASE WHEN activity.ID IS NOT NULL THEN sr.ID END) * 100.0 / COUNT(DISTINCT sr.ID), 2) as activity_rate,
    COUNT(activity.ID) as total_activities,
    ROUND(AVG(CASE WHEN activity.ID IS NOT NULL THEN 1.0 * (SELECT COUNT(*) FROM activity a2 WHERE a2.SR_ID = sr.ID) END), 2) as avg_activities_per_sr,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)), 2) as avg_activity_hours,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)) / 24, 1) as avg_activity_days
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
LEFT JOIN activity ON sr.ID = activity.SR_ID
WHERE strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY c.ID, c.NAME
HAVING COUNT(DISTINCT sr.ID) > 100
ORDER BY total_srs DESC
LIMIT 20
""", conn)

print("="*70)
print("📋 ANALYSE DES ACTIVITÉS PAR CATÉGORIE")
print("="*70)
print("\nIndicateurs :")
print("  • Total SRs : Nombre total de demandes")
print("  • SRs avec activité : Nombre de SRs nécessitant une intervention manuelle")
print("  • Taux d'activité : % de SRs nécessitant une activité")
print("  • Activités moyennes : Nombre moyen d'activités par SR")
print("  • Temps moyen activité : Durée moyenne de traitement d'une activité\n")

display(activity_analysis)

# Statistiques globales
total_with_activity = activity_analysis['srs_with_activity'].sum()
total_all = activity_analysis['total_srs'].sum()
global_activity_rate = (total_with_activity / total_all) * 100
avg_activity_time = activity_analysis['avg_activity_hours'].mean()

print(f"\n📊 Statistiques globales (Top 20 catégories) :")
print(f"   • SRs avec activité : {total_with_activity:,} / {total_all:,}")
print(f"   • Taux global d'activité : {global_activity_rate:.2f}%")
print(f"   • Temps moyen d'une activité : {avg_activity_time:.1f}h ({avg_activity_time/24:.1f}j)")

📋 ANALYSE DES ACTIVITÉS PAR CATÉGORIE

Indicateurs :
  • Total SRs : Nombre total de demandes
  • SRs avec activité : Nombre de SRs nécessitant une intervention manuelle
  • Taux d'activité : % de SRs nécessitant une activité
  • Activités moyennes : Nombre moyen d'activités par SR
  • Temps moyen activité : Durée moyenne de traitement d'une activité



,category_name,total_srs,srs_with_activity,activity_rate,total_activities,avg_activities_per_sr,avg_activity_hours,avg_activity_days
0,Tax,103527,8129,7.85,17314,9.13,89.87,3.7
1,Cash instruction,91925,6579,7.16,8855,2.31,1.58,0.1
2,BAU Asset Creation,82196,7,0.01,11,2.27,3.72,0.2
3,CREST,76884,33398,43.44,40788,1.84,1.02,0.0
4,Others,75916,1865,2.46,2274,2.26,134.45,5.6
5,Investigation Level 1,71533,2752,3.85,3371,1.60,1.05,0.0
6,Settlement,65129,55,0.08,69,1.67,192.05,8.0
7,FRPP,63285,6766,10.69,9734,2.40,0.42,0.0
8,ESMX,60882,6928,11.38,7405,1.19,0.65,0.0
9,DEFF,58473,7340,12.55,10090,1.97,0.29,0.0



📊 Statistiques globales (Top 20 catégories) :
   • SRs avec activité : 86,678 / 1,235,603
   • Taux global d'activité : 7.02%
   • Temps moyen d'une activité : 24.8h (1.0j)


In [38]:
# Visualisation : Taux d'activité par catégorie
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Taux d\'Activité par Catégorie (Top 15)', 'Volume SRs vs Taux d\'Activité'),
    specs=[[{"type": "bar"}, {"type": "scatter"}]]
)

df_viz = activity_analysis.head(15).copy()
# Remplacer les NaN par 1 pour éviter les erreurs dans la taille des points
df_viz['avg_activities_per_sr'] = df_viz['avg_activities_per_sr'].fillna(1)

# Graphique 1 : Barres horizontales du taux d'activité
fig.add_trace(
    go.Bar(
        y=df_viz['category_name'],
        x=df_viz['activity_rate'],
        orientation='h',
        text=df_viz['activity_rate'].apply(lambda x: f"{x:.1f}%"),
        textposition='outside',
        marker_color=df_viz['activity_rate'],
        marker_colorscale='Blues',
        name='Taux activité',
        hovertemplate='<b>%{y}</b><br>Taux: %{x:.1f}%<extra></extra>'
    ),
    row=1, col=1
)

# Graphique 2 : Scatter volume vs taux d'activité
fig.add_trace(
    go.Scatter(
        x=df_viz['total_srs'],
        y=df_viz['activity_rate'],
        mode='markers+text',
        marker=dict(
            size=df_viz['avg_activities_per_sr']*10,
            color=df_viz['activity_rate'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Taux<br>Activité", x=1.15)
        ),
        text=df_viz['category_name'].str[:15],
        textposition='top center',
        hovertemplate='<b>%{text}</b><br>Volume: %{x:,}<br>Taux activité: %{y:.1f}%<extra></extra>',
        name='Catégories'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Taux d'Activité (%)", row=1, col=1)
fig.update_yaxes(title_text="Catégorie", row=1, col=1)
fig.update_xaxes(title_text="Nombre de SRs", row=1, col=2)
fig.update_yaxes(title_text="Taux d'Activité (%)", row=1, col=2)

fig.update_layout(height=500, showlegend=False, title_text="Analyse des Activités par Catégorie")
fig.show()

---
## 10. Analyse Détaillée des Activités

**Focus** : Statistiques sur les activités elles-mêmes (types, durées, volumes)

In [39]:
# Statistiques globales sur les activités
activity_stats = pd.read_sql_query("""
SELECT 
    COUNT(*) as total_activities,
    COUNT(DISTINCT SR_ID) as unique_srs_with_activities,
    COUNT(CASE WHEN CLOSINGDATE IS NOT NULL THEN 1 END) as closed_activities,
    ROUND(COUNT(CASE WHEN CLOSINGDATE IS NOT NULL THEN 1 END) * 100.0 / COUNT(*), 2) as closure_rate,
    ROUND(AVG(CAST((julianday(CLOSINGDATE) - julianday(CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close,
    ROUND(MIN(CAST((julianday(CLOSINGDATE) - julianday(CREATIONDATE)) * 24 AS REAL)), 2) as min_hours,
    ROUND(MAX(CAST((julianday(CLOSINGDATE) - julianday(CREATIONDATE)) * 24 AS REAL)), 2) as max_hours
FROM activity
WHERE strftime('%Y-%m', CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
""", conn)

print("="*70)
print("📊 STATISTIQUES GLOBALES DES ACTIVITÉS")
print("="*70)
print(f"\n📋 Total d'activités : {activity_stats['total_activities'][0]:,}")
print(f"🎯 SRs concernés : {activity_stats['unique_srs_with_activities'][0]:,}")
print(f"✅ Activités fermées : {activity_stats['closed_activities'][0]:,} ({activity_stats['closure_rate'][0]}%)")
print(f"⏱️  Temps moyen de traitement : {activity_stats['avg_hours_to_close'][0]:.1f}h ({activity_stats['avg_hours_to_close'][0]/24:.1f}j)")
print(f"⚡ Temps min : {activity_stats['min_hours'][0]:.2f}h | Temps max : {activity_stats['max_hours'][0]:.1f}h ({activity_stats['max_hours'][0]/24:.0f}j)")

display(activity_stats)

📊 STATISTIQUES GLOBALES DES ACTIVITÉS

📋 Total d'activités : 341,838
🎯 SRs concernés : 241,236
✅ Activités fermées : 338,158 (98.92%)
⏱️  Temps moyen de traitement : 42.1h (1.8j)
⚡ Temps min : 0.00h | Temps max : 11450.3h (477j)


,total_activities,unique_srs_with_activities,closed_activities,closure_rate,avg_hours_to_close,min_hours,max_hours
0,341838,241236,338158,98.92,42.11,0.0,11450.29


In [40]:
# Analyse par type d'activité (par TYPE_ID)
activity_by_type = pd.read_sql_query("""
SELECT 
    COALESCE(activity.TYPE_ID, 0) as type_id,
    COUNT(activity.ID) as total_activities,
    COUNT(DISTINCT activity.SR_ID) as unique_srs,
    ROUND(COUNT(activity.ID) * 100.0 / (SELECT COUNT(*) FROM activity WHERE strftime('%Y-%m', CREATIONDATE) BETWEEN '2024-01' AND '2025-09'), 2) as pct_of_total,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)) / 24, 1) as avg_days_to_close
FROM activity
WHERE strftime('%Y-%m', activity.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY activity.TYPE_ID
ORDER BY total_activities DESC
LIMIT 20
""", conn)

# Ajouter un nom lisible
activity_by_type['type_name'] = 'Type ' + activity_by_type['type_id'].astype(str)

print("="*70)
print("📊 ACTIVITÉS PAR TYPE (Top 20)")
print("="*70)
print(f"\nTop 5 types représentent {activity_by_type.head(5)['pct_of_total'].sum():.1f}% des activités\n")

display(activity_by_type)

📊 ACTIVITÉS PAR TYPE (Top 20)

Top 5 types représentent 100.0% des activités



,type_id,total_activities,unique_srs,pct_of_total,avg_hours_to_close,avg_days_to_close,type_name
0,1579,341424,240911,99.88,41.98,1.7,Type 1579
1,1580,413,413,0.12,171.33,7.1,Type 1580
2,1582,1,1,0.00,0.32,0.0,Type 1582


In [41]:
# Analyse par statut d'activité (par STATUS_ID)
activity_by_status = pd.read_sql_query("""
SELECT 
    COALESCE(activity.STATUS_ID, 0) as status_id,
    COUNT(activity.ID) as total_activities,
    COUNT(DISTINCT activity.SR_ID) as unique_srs,
    ROUND(COUNT(activity.ID) * 100.0 / (SELECT COUNT(*) FROM activity WHERE strftime('%Y-%m', CREATIONDATE) BETWEEN '2024-01' AND '2025-09'), 2) as pct_of_total,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)) / 24, 1) as avg_days_to_close
FROM activity
WHERE strftime('%Y-%m', activity.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY activity.STATUS_ID
ORDER BY total_activities DESC
LIMIT 20
""", conn)

# Ajouter un nom lisible
activity_by_status['status_name'] = 'Status ' + activity_by_status['status_id'].astype(str)

print("="*70)
print("📊 ACTIVITÉS PAR STATUT (Top 20)")
print("="*70)
print(f"\nTop 5 statuts représentent {activity_by_status.head(5)['pct_of_total'].sum():.1f}% des activités\n")

display(activity_by_status)

📊 ACTIVITÉS PAR STATUT (Top 20)

Top 5 statuts représentent 100.0% des activités



,status_id,total_activities,unique_srs,pct_of_total,avg_hours_to_close,avg_days_to_close,status_name
0,31,324637,232597,94.97,36.48,1.5,Status 31
1,11,13443,11643,3.93,174.20,7.3,Status 11
2,100,3074,2718,0.90,4.55,0.2,Status 100
3,10,549,479,0.16,1091.24,45.5,Status 10
4,12,118,85,0.03,580.07,24.2,Status 12
5,101,11,10,0.00,0.11,0.0,Status 101
6,13,3,3,0.00,0.06,0.0,Status 13
7,14,3,3,0.00,0.05,0.0,Status 14


In [42]:
# Évolution mensuelle des activités
monthly_activities = pd.read_sql_query("""
SELECT 
    strftime('%Y-%m', activity.CREATIONDATE) as month,
    COUNT(activity.ID) as total_activities,
    COUNT(DISTINCT activity.SR_ID) as unique_srs,
    COUNT(CASE WHEN activity.CLOSINGDATE IS NOT NULL THEN 1 END) as closed_activities,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close
FROM activity
WHERE strftime('%Y-%m', activity.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY month
ORDER BY month
""", conn)

print("="*70)
print("📈 ÉVOLUTION MENSUELLE DES ACTIVITÉS")
print("="*70)
print(f"\nPériode : Janvier 2024 - Septembre 2025")
print(f"Nombre de mois : {len(monthly_activities)}\n")

display(monthly_activities)

# Visualisation de la tendance
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Volume d\'Activités par Mois', 'Temps Moyen de Traitement par Mois'),
    vertical_spacing=0.15
)

# Graphique 1 : Volume mensuel
fig.add_trace(
    go.Scatter(
        x=monthly_activities['month'],
        y=monthly_activities['total_activities'],
        mode='lines+markers',
        name='Activités',
        line=dict(color='royalblue', width=3),
        marker=dict(size=8),
        hovertemplate='<b>%{x}</b><br>Activités: %{y:,}<extra></extra>'
    ),
    row=1, col=1
)

# Graphique 2 : Temps moyen mensuel
fig.add_trace(
    go.Scatter(
        x=monthly_activities['month'],
        y=monthly_activities['avg_hours_to_close'] / 24,
        mode='lines+markers',
        name='Temps moyen',
        line=dict(color='coral', width=3),
        marker=dict(size=8),
        hovertemplate='<b>%{x}</b><br>Temps: %{y:.1f} jours<extra></extra>'
    ),
    row=2, col=1
)

fig.update_xaxes(title_text="Mois", row=2, col=1)
fig.update_yaxes(title_text="Nombre d'Activités", row=1, col=1)
fig.update_yaxes(title_text="Jours", row=2, col=1)

fig.update_layout(height=700, showlegend=False, title_text="Tendances Temporelles des Activités")
fig.show()

📈 ÉVOLUTION MENSUELLE DES ACTIVITÉS

Période : Janvier 2024 - Septembre 2025
Nombre de mois : 21



,month,total_activities,unique_srs,closed_activities,avg_hours_to_close
0,2024-01,14525,10773,14397,51.96
1,2024-02,14070,10620,13947,65.27
2,2024-03,15251,11293,15145,46.21
3,2024-04,17944,13062,17800,38.68
4,2024-05,19377,14188,19232,43.69
5,2024-06,16775,12505,16623,43.86
6,2024-07,18286,13582,18115,49.51
7,2024-08,14838,11152,14719,49.04
8,2024-09,14543,10850,14426,38.84
9,2024-10,17189,12537,17039,39.93


In [43]:
# Visualisation : Distribution des activités
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Volume d\'Activités par Statut (Top 10)', 'Temps de Traitement par Type (Top 10)'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

# Graphique 1 : Volume par statut
df_status_top = activity_by_status.head(10).sort_values('total_activities')
fig.add_trace(
    go.Bar(
        y=df_status_top['status_name'],
        x=df_status_top['total_activities'],
        orientation='h',
        text=df_status_top['total_activities'].apply(lambda x: f"{x:,}"),
        textposition='outside',
        marker_color='lightblue',
        name='Volume',
        hovertemplate='<b>%{y}</b><br>Volume: %{x:,}<extra></extra>'
    ),
    row=1, col=1
)

# Graphique 2 : Temps par type
df_type_top = activity_by_type.head(10).sort_values('avg_days_to_close')
fig.add_trace(
    go.Bar(
        y=df_type_top['type_name'],
        x=df_type_top['avg_days_to_close'],
        orientation='h',
        text=df_type_top['avg_days_to_close'].apply(lambda x: f"{x:.1f}j"),
        textposition='outside',
        marker_color=df_type_top['avg_days_to_close'],
        marker_colorscale='Reds',
        name='Durée',
        hovertemplate='<b>%{y}</b><br>Durée: %{x:.1f} jours<extra></extra>'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Nombre d'Activités", row=1, col=1)
fig.update_yaxes(title_text="Statut", row=1, col=1)
fig.update_xaxes(title_text="Jours", row=1, col=2)
fig.update_yaxes(title_text="Type", row=1, col=2)

fig.update_layout(height=500, showlegend=False, title_text="Distribution des Activités")
fig.show()

---
## 11. Synthèse


In [44]:
print("="*70)
print("📋 SYNTHÈSE EXÉCUTIVE")
print("="*70)

# Top 3 catégories par volume
print("\n1️⃣ TOP 3 CATÉGORIES PAR VOLUME (Focus d'amélioration)")
for idx, row in category_distribution.head(3).iterrows():
    print(f"   • {row['category_name']}: {row['total_requests']:,} demandes ({row['pct_of_total']}%)")
    print(f"     → Temps moyen: {row['avg_hours_to_close']/24:.1f} jours | Closure: {row['closure_rate']}%")

# Catégories les plus lentes
print("\n2️⃣ CATÉGORIES AVEC TEMPS DE TRAITEMENT LE PLUS LONG (Pain Points)")
for idx, row in treatment_time_by_category.head(3).iterrows():
    print(f"   • {row['category_name']}: {row['avg_days']:.1f} jours en moyenne")
    print(f"     → Volume: {row['total_requests']:,} demandes")

# KPIs moyens
avg_resolution = client_experience_kpis['resolution_rate'].mean()
avg_sla = client_experience_kpis['sla_compliance'].mean()
avg_first_response = client_experience_kpis['avg_first_response_hours'].mean()

print("\n3️⃣ KPIs GLOBAUX EXPÉRIENCE CLIENT")
print(f"   • Taux de résolution moyen: {avg_resolution:.1f}%")
print(f"   • SLA Compliance moyen: {avg_sla:.1f}%")
print(f"   • Temps de première réponse: {avg_first_response:.1f}h ({avg_first_response/24:.1f}j)")

📋 SYNTHÈSE EXÉCUTIVE

1️⃣ TOP 3 CATÉGORIES PAR VOLUME (Focus d'amélioration)
   • Tax: 103,527 demandes (2.5%)
     → Temps moyen: 19.1 jours | Closure: 97.25%
   • Cash instruction: 91,925 demandes (2.22%)
     → Temps moyen: 2.2 jours | Closure: 99.89%
   • BAU Asset Creation: 82,196 demandes (1.98%)
     → Temps moyen: 0.9 jours | Closure: 99.97%

2️⃣ CATÉGORIES AVEC TEMPS DE TRAITEMENT LE PLUS LONG (Pain Points)
   • Change And Transitions: 122.2 jours en moyenne
     → Volume: 108 demandes
   • EXTERNAL QUERIES: 103.6 jours en moyenne
     → Volume: 7,664 demandes
   • Post-Reporting 1042-S: 87.0 jours en moyenne
     → Volume: 106 demandes

3️⃣ KPIs GLOBAUX EXPÉRIENCE CLIENT
   • Taux de résolution moyen: 99.6%
   • SLA Compliance moyen: 73.8%
   • Temps de première réponse: 1.3h (0.1j)


---
## 12. Export des Résultats

Sauvegarder les analyses pour présentation.

In [45]:
# Export des analyses clés
timestamp = datetime.now().strftime('%Y%m%d')

# 1. Répartition par catégorie
export_1 = EXPORT_DIR / f'category_distribution_{timestamp}.csv'
category_distribution.to_csv(export_1, index=False)
print(f"✅ Exporté: {export_1.name}")

# 2. Temps de traitement
export_2 = EXPORT_DIR / f'treatment_time_by_category_{timestamp}.csv'
treatment_time_by_category.to_csv(export_2, index=False)
print(f"✅ Exporté: {export_2.name}")

# 3. KPIs expérience client
export_3 = EXPORT_DIR / f'client_experience_kpis_{timestamp}.csv'
client_experience_kpis.to_csv(export_3, index=False)
print(f"✅ Exporté: {export_3.name}")

# 4. Pain points
if len(pain_points) > 0:
    export_4 = EXPORT_DIR / f'pain_points_{timestamp}.csv'
    pain_points.to_csv(export_4, index=False)
    print(f"✅ Exporté: {export_4.name}")

# 5. Tendances mensuelles
export_5 = EXPORT_DIR / f'monthly_category_trends_{timestamp}.csv'
monthly_category_trends.to_csv(export_5, index=False)
print(f"✅ Exporté: {export_5.name}")

# 6. Analyse des activités
export_6 = EXPORT_DIR / f'activity_analysis_{timestamp}.csv'
activity_analysis.to_csv(export_6, index=False)
print(f"✅ Exporté: {export_6.name}")

# 7. Activités par statut
export_7 = EXPORT_DIR / f'activity_by_status_{timestamp}.csv'
activity_by_status.to_csv(export_7, index=False)
print(f"✅ Exporté: {export_7.name}")

# 8. Activités par type
export_8 = EXPORT_DIR / f'activity_by_type_{timestamp}.csv'
activity_by_type.to_csv(export_8, index=False)
print(f"✅ Exporté: {export_8.name}")

# 9. Évolution mensuelle des activités
export_9 = EXPORT_DIR / f'monthly_activities_{timestamp}.csv'
monthly_activities.to_csv(export_9, index=False)
print(f"✅ Exporté: {export_9.name}")

print(f"\n📂 Tous les exports dans: {EXPORT_DIR}")

✅ Exporté: category_distribution_20260212.csv
✅ Exporté: treatment_time_by_category_20260212.csv
✅ Exporté: client_experience_kpis_20260212.csv
✅ Exporté: pain_points_20260212.csv
✅ Exporté: monthly_category_trends_20260212.csv
✅ Exporté: activity_analysis_20260212.csv
✅ Exporté: activity_by_status_20260212.csv
✅ Exporté: activity_by_type_20260212.csv
✅ Exporté: monthly_activities_20260212.csv

📂 Tous les exports dans: /Users/jo/Desktop/BNP Paribas/Data/Exports


In [46]:
# Fermer la connexion
conn.close()
print("✅ Connexion fermée")

✅ Connexion fermée
